# Leveraging remote sensing to explore spatio-temporal dynamics of channel extent and migration in Himalayan lowlands : Response and implications to monsoon flood

In [ ]:
import ee
import geemap
import hydrafloods as hf
from rivgraph.classes import river
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union, polygonize

In [ ]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## River morphology

In [ ]:
roi = ee.FeatureCollection('users/Nischal_Karki/Bagmati_area').geometry()
mesh = ee.FeatureCollection('users/Nischal_Karki/mesh')

bn8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B6', 'QA_PIXEL', 'SR_B5', 'SR_B7']
bn7 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5', 'QA_PIXEL', 'SR_B4', 'SR_B7']
bn5 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B5', 'QA_PIXEL', 'SR_B4', 'SR_B7']
bns = ['uBlue', 'Blue', 'Green', 'Red', 'Swir1', 'BQA', 'Nir', 'Swir2']

ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(bn5, bns)
ls7 = (ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").select(bn7, bns))
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(bn8, bns)
merged = ls5.merge(ls7).merge(ls8)

def Ndvi(image):
    return image.normalizedDifference(['Nir', 'Red']).rename('ndvi')

def Lswi(image):
    return image.normalizedDifference(['Nir', 'Swir1']).rename('lswi')

def Mndwi(image):
    return image.normalizedDifference(['Green', 'Swir1']).rename('mndwi')

def Evi(image):
    evi = image.expression('2.5 * (Nir - Red) / (1 + Nir + 6 * Red - 7.5 * Blue)', {
    'Nir': image.select(['Nir']),
    'Red': image.select(['Red']),
    'Blue': image.select(['Blue'])
    })
    return evi

def qa(image):
    qa = image.select('BQA')
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2).multiply(0.0000275).add(-0.2).clip(roi)

bnp50 = ['uBlue_p50', 'Blue_p50', 'Green_p50', 'Red_p50', 'Swir1_p50', 'BQA_p50', 'Nir_p50', 'Swir2_p50']

def get_mask(imgCol,mndwi_param = -0.35,ndvi_param = 0.15,cleaning_pixels = 100):
    p50 = imgCol.reduce(ee.Reducer.percentile([50])).select(bnp50, bns)
    
    # Apply to each percentile
    mndwi_p50 = Mndwi(p50)
    ndvi_p50 = Ndvi(p50)
    evi_p50 = Evi(p50)
    lswi_p50 = Lswi(p50)
    
    #Water classification from (Zou 2018):
    water_p50 = (mndwi_p50.gt(ndvi_p50).Or(mndwi_p50.gt(evi_p50))).And(evi_p50.lt(0.1))
    waterMasked_p50 = water_p50.updateMask(water_p50.gt(0))
    
    #Active river belt classification:
    activebelt_p50 = (mndwi_p50.gte(mndwi_param)).And(ndvi_p50.lte(ndvi_param))
    activebeltMasked_p50 = activebelt_p50.updateMask(activebelt_p50.gt(0))
    active_p50 = (water_p50).Or(activebelt_p50)
    
    #Clean binary active channel:
    smooth_map_p50 = active_p50.focal_mode(radius= 10, kernelType= 'octagon', units= 'pixels', iterations= 1).mask(active_p50.gte(1));
    
    noise_removal_p50 = active_p50.updateMask(active_p50.connectedPixelCount(cleaning_pixels, False).gte(cleaning_pixels)).unmask(smooth_map_p50);
    
    #noise_removal_p50_Masked = noise_removal_p50.updateMask(noise_removal_p50.gt(0))
    
    return noise_removal_p50

year=ee.List.sequence(1990,2022)

# For downloading annual channel masks
def get_annual(year):
    imgCol = merged.filterBounds(roi).filter(ee.Filter.calendarRange(year, year, 'year')).map(qa)
    wm = get_mask(imgCol)
    return wm.set({'year':year})

annual_masks = ee.ImageCollection(year.map(get_annual))
geemap.ee_export_image_collection_to_drive(annual_masks, scale=10, folder = "RivMask", maxPixels=1e10, crs='EPSG:32645',region=roi)

# For erosion-accretion areas
def calc_area(year):
    imgCol1 = merged.filterBounds(roi).filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(1, 5, 'month')).map(qa)
    imgCol2 = merged.filterBounds(roi).filter(ee.Filter.calendarRange(year, year, 'year')).filter(ee.Filter.calendarRange(6, 12, 'month')).map(qa)
    wm1 = get_mask(imgCol1)
    wm2 = get_mask(imgCol2)
    diff = wm1.subtract(wm2)
    E = diff.eq(1)
    A = diff.eq(-1)
    A_E = A.rename('Accretion').addBands(E.rename('Erosion'))
    
    def get_stat(img):
        area = img.selfMask().multiply(ee.Image.pixelArea()).reduceRegions(collection=mesh,
                                                                reducer=ee.Reducer.sum(),
                                                                scale=30)
        return area.set({'year':year})
    
    return get_stat(wm)

def set_year(fc):
    year = ee.FeatureCollection(fc).get('year')
    fc_year = ee.FeatureCollection(fc).map(lambda x: x.set({'year':year}))
    return fc_year

task = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection(ee.FeatureCollection(test.map(set_year)).flatten()), description='total_area',fileFormat='CSV')
task.start()

## Centerline migration
Download annual channel masks to a folder for centerline delineation. Change `path` to your folder path.

In [ ]:
path = "/Users/nischalkarki/Bagmati/RivMask/"
full_path = glob.glob(f'{path}/*.tif')

for file in full_path:
    try:
        mask_path = file
        name = file[-8:-4]
        results_folder = "/Users/nischalkarki/Bagmati"
        es = 'NS'
        riv = river(name,mask_path,results_folder,exit_sides=es,verbose=True)
        riv.skeletonize()
        riv.compute_network()
        riv.prune_network()
        riv.compute_link_width_and_length()
        riv.compute_mesh()
    
    except:
        riv.to_geovectors('centerline',ftype='json')

mesh = gpd.read_file('/Users/nischalkarki/Bagmati_mesh.shp').to_crs('EPSG:32645')
result = gpd.GeoDataFrame()

for year in range(1990,2022):
    ref_cl = gpd.read_file('/Users/nischalkarki/Bagmati/2022_centerline.json')
    ref_cl.crs = 'EPSG:32645'
    
    cur_cl = gpd.read_file(f'/Users/nischalkarki/Bagmati/{year}_centerline.json')
    cur_cl.crs = 'EPSG:32645'
    next_cl = gpd.read_file(f'/Users/nischalkarki/Bagmati/{year+1}_centerline.json')
    next_cl.crs = 'EPSG:32645'
    
    ref_cline = ref_cl.overlay(mesh,how="intersection")
    ref_cline = ref_cline[['cngmeters','geometry']]
    
    cur_cline = cur_cl.overlay(mesh,how="intersection")
    cur_cline['length']=cur_cline.geometry.apply(lambda geom: geom.length)
    cur_cline['year']=year
    cur_cline = cur_cline[['cngmeters','length','year']]
    
    # Extract coordinates from the GeoJSON geometries
    line1_coords = list(cur_cl.geometry.iloc[0].coords)
    line2_coords = list(next_cl.geometry.iloc[0].coords)
    
    # Reverse the order of line2_coords for the calculation
    line2_coords.reverse()
    
    # Combine the coordinates and close the loop
    combined_coords = line1_coords + line2_coords + [line1_coords[0]]
    
    # Create a LineString geometry
    combined_polyline = LineString(combined_coords)
    mls = unary_union(combined_polyline)
    
    # Polygonize the unioned LineString
    polygons = list(polygonize(mls))
    
    # Create a GeoDataFrame from the polygons
    gdf = gpd.GeoDataFrame(geometry=polygons, crs='EPSG:32645')
    
    area_trav = gdf.dissolve().overlay(mesh,how='intersection').sjoin(mesh, how='left', predicate='within').rename({'cngmeters_left':'cngmeters'},axis='columns')
    area_trav['area']=area_trav.geometry.apply(lambda geom: geom.area)
    area_trav=area_trav[['cngmeters','area']]
    
    final = ref_cline.merge(area_trav,on='cngmeters').merge(cur_cline,on='cngmeters')
    result = pd.concat([result,final])
    
result.to_csv('/Users/nischalkarki/Bagmati/migration.csv')

## Sentinel-1 flood mapping

In [ ]:
def border_noise(img):
    '''
    Removes border noise from s1 image;
    '''
    binary = img.select('VV').lt(-35).rename("binary")
    connected = binary.connectedComponents(ee.Kernel.plus(1), 128).select("labels").Not()
    mask = binary.where(connected.eq(0),0).Not()
    kernel = ee.Kernel.square(5)
    mask = mask.reduceNeighborhood(reducer=ee.Reducer.And(), kernel=kernel)
    result= img.updateMask(mask)
    return result

band="VV"

s1Collection = ee.ImageCollection('COPERNICUS/S1_GRD') \ 
.filterBounds(roi).filterDate('2019-07-13','2019-07-14') \
.filter(ee.Filter.eq('instrumentMode', 'IW')) \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
.filter(ee.Filter.eq('resolution_meters',10)) \
.select(['angle','VV']).map(border_noise)

# MERIT Hydro dataset, a hydrologically adjusted DEM from Yamazaki, D. et al. 2019
# http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro/

merit = ee.Image("MERIT/Hydro/v1_0_1")
dem = merit.select("elv").unmask(0)
hand = merit.select("hnd").unmask(0)

s1 = hf.Dataset.from_imgcollection(s1Collection)  # ee.ImageCollection to hf.datasets
s1_flat = s1.apply_func(hf.slope_correction, elevation=dem, buffer=100) # Terrain flattening
s1_filtered = s1_flat.apply_func(hf.refined_lee) # Speckle filtering

water = s1_filtered.apply_func(
    hf.thresholding.edge_otsu,
    initial_threshold=-16,
    thresh_no_data=-20,
    edge_buffer=300
)

# Extract permanent water from Dynamic World dataset - near real time global landcover from sentinel-2
dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate('2019-01-01', '2019-12-31')
dwImage = dwCol.select('label').mode()
dw_mask = dwImage.eq(0)

def post_process(img):
    final = (
    hf.close_binary(
        hf.open_binary(img, window=1.5) # apply opening filter
        .updateMask(img.mask()), # force mask to be consistent with sar imagery
        window=1.5
    ) # apply closing filter
    .updateMask(img.mask()) # force mask to be consistent with sar imagery
    .updateMask(hand.lt(15)) # only pixels that were originally classified as water AND < 15m from HAND (Height Above Nearest Drainage)
    .updateMask(dw_mask.Not()) # remove permanent water
    )
    connected = final.selfMask().connectedPixelCount()
    final_img = final.selfMask().updateMask(connected.gte(10))
    return final_img.selfMask()

water_final = water.collection.map(post_process)

task = ee.batch.Export.image.toDrive(image=water_final, region = roi, crs="EPSG:32645",scale=10, maxPixels=1e10,description= "Bagmati_flood")
task.start()
